***Import statements***

In [ ]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import csv
from io import StringIO

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


***Input the query from user***

In [ ]:
test_str = input()

print("The original string is : " + test_str)
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

for ele in test_str:
    if ele in punc:
        test_str = test_str.replace(ele, "")
print("The string after punctuation filter : " + test_str)


need flights from bangalore to kolkata
The original string is : need flights from bangalore to kolkata
The string after punctuation filter : need flights from bangalore to kolkata


***Extract source and destination from query***

In [ ]:
test_str_arr = [x.strip() for x in test_str.strip('[]').split(' ') ]
index = test_str_arr.index("to")
src = test_str_arr[index-1]
destn = test_str_arr[index+1] 

# src = src.capitalize()
# destn = destn.capitalize()
print(src)
print(destn)

bangalore
kolkata


***Main scrap - flight from and to the given cities - from GoIbibo***

In [ ]:
base_url='https://www.goibibo.com/flights/'
base_url = base_url + "/{}-to-{}-flights/"
url = base_url.format(src, destn)

r = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')
link = ""
whole_str = ''
for i in soup.findAll('div', {'class': 'srp-card-uistyles__CardLeft-sc-3flq99-5 hklZXY'}):
    for data in i.find_all('span'):
      whole_str+=(data.text)
      
      # print(whole_str)
    link+=(whole_str+"\n")
    whole_str=''
    # link.append("\n")
# print(remove_tags(str(link)))
# link = link.replace(" ", "")
print(link)

Air IndiaBLRBengaluru, India18:45-HYD-HYD-13h 20mCCUKolkata, India08:05
Air IndiaBLRBengaluru, India05:45 2h 35mCCUKolkata, India08:20
IndiGoBLRBengaluru, India18:30-MAA-MAA-4h 50mCCUKolkata, India23:20
IndiGoBLRBengaluru, India09:15-MAA-MAA-5h 30mCCUKolkata, India14:45
IndiGoBLRBengaluru, India17:10-MAA-MAA-6h 10mCCUKolkata, India23:20
IndiGoBLRBengaluru, India15:10-MAA-MAA-6h 20mCCUKolkata, India21:30
IndiGoBLRBengaluru, India08:15-MAA-MAA-6h 30mCCUKolkata, India14:45
IndiGoBLRBengaluru, India07:10-MAA-MAA-7h 35mCCUKolkata, India14:45
IndiGoBLRBengaluru, India23:05-HYD-HYD-7h 40mCCUKolkata, India06:45
IndiGoBLRBengaluru, India06:05-MAA-MAA-8h 40mCCUKolkata, India14:45



In [ ]:
link = StringIO(link)
df = pd.read_csv(link, sep ="\n")
print(df)

  Air IndiaBLRBengaluru, India18:45-HYD-HYD-13h 20mCCUKolkata, India08:05
0  Air IndiaBLRBengaluru, India05:45 2h 35mCCUKol...                     
1  IndiGoBLRBengaluru, India18:30-MAA-MAA-4h 50mC...                     
2  IndiGoBLRBengaluru, India09:15-MAA-MAA-5h 30mC...                     
3  IndiGoBLRBengaluru, India17:10-MAA-MAA-6h 10mC...                     
4  IndiGoBLRBengaluru, India15:10-MAA-MAA-6h 20mC...                     
5  IndiGoBLRBengaluru, India08:15-MAA-MAA-6h 30mC...                     
6  IndiGoBLRBengaluru, India07:10-MAA-MAA-7h 35mC...                     
7  IndiGoBLRBengaluru, India23:05-HYD-HYD-7h 40mC...                     
8  IndiGoBLRBengaluru, India06:05-MAA-MAA-8h 40mC...                     


In [ ]:
file_name = "/content/gdrive/My Drive/Colab Notebooks/IR_SearchEngine_Package/{}-to-{}-flights.csv"
# with open(file_name.format(src, destn), "a") as outfile:
#     df.writ
# outfile.close()

df.to_csv(file_name.format(src, destn))

***Cheap fares***


In [ ]:
doc = lh.fromstring(r.content)
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Bangalore Kolkata Cheap Flights"
2:"Fare"
3:"Date"


In [ ]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

cheap_flights_dict={title:column for (title,column) in col}
# df=pd.DataFrame(Dict)
# df.head(5)
print(cheap_flights_dict)

{'Bangalore Kolkata Cheap Flights': ['Lowest Airfare Today', 'Cheapest Fare Within 30 Days', 'Cheapest Fare Within 90 Days', 'Cheapest Fare Within 365 Days'], 'Fare': ['Rs 5176', 'Rs 3981', 'Rs 3215', 'Rs 2977'], 'Date': ['01 Oct', '31 Oct', '14 Nov', '01 Feb']}


**Recommendation**

In [211]:
def remove_tags(html):
    soup = BeautifulSoup(html, 'html.parser')
    for data in soup(['span']):
        # Remove tags
        data.decompose()
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [212]:
 # soup.find('div', {'class':'footer-wrapper_links'}).find_all('a')
recommendations = []
for i in soup.find('div', {'class':'common-stylesstyles__CommonDetails-sc-uferee-1 list-for-linksstyles__Details-sc-18mazeo-0 ikmQXy jjVksl'}):
    # print(i)
    recommendations.append(i)
# print(link)

In [213]:
recommendations = remove_tags(str(recommendations))
recommendations

'[ Further Flights from Bangalore (BLR) , Bangalore to Surat Flights , Bangalore to Patna Flights , Bangalore to Chennai Flights , Bangalore to Delhi Flights , Bangalore to Hyderabad Flights , Bangalore to Goa Flights , Bangalore to Mumbai Flights , Bangalore to Aurangabad Flights , Bangalore to Pantnagar Flights , Bangalore to Satna Flights , Bangalore to Tiruchirappalli Flights , Bangalore to Ratnagiri Flights , Bangalore to Nasik Flights , Bangalore to Aizawl Flights , Bangalore to Jabalpur Flights ]'

In [ ]:
towrite = list()
towrite.append(recommendations)
towrite.append(cheap_flights_dict)
file_name = "/content/gdrive/My Drive/Colab Notebooks/IR_SearchEngine_Package/{}-to-{}-further.csv"
with open(file_name.format(src, destn), "a") as outfile:
    outfile.write(str(towrite))
outfile.close()

In [ ]:
with open(file_name.format(src, destn), "r") as infile:
    line = infile.readline()
    print(line)

['[ Further Flights from Kolkata (CCU) , Kolkata to Surat Flights , Kolkata to Patna Flights , Kolkata to Chennai Flights , Kolkata to Delhi Flights , Kolkata to Hyderabad Flights , Kolkata to Goa Flights , Kolkata to Bangalore Flights , Kolkata to Vadodara Flights , Kolkata to Tiruchirappalli Flights , Kolkata to Bhavnagar Flights , Kolkata to Port Blair Flights , Kolkata to Hubli Flights , Kolkata to Bikaner Flights , Kolkata to Puttaparthi Flights , Kolkata to Bellary Flights ]', {'Bangalore Kolkata Cheap Flights': ['Lowest Airfare Today', 'Cheapest Fare Within 30 Days', 'Cheapest Fare Within 90 Days', 'Cheapest Fare Within 365 Days'], 'Fare': ['Rs 5176', 'Rs 3981', 'Rs 3215', 'Rs 2977'], 'Date': ['01 Oct', '31 Oct', '14 Nov', '01 Feb']}]


**JSON** - mostly no need (might delete as well)

In [ ]:
# # /*merge all documents into a single file - datastore */
# import json
  
# # Serializing json 
# json_object = json.dumps(cheap_flights_dict, indent = 4)
# json_o2 = json.dumps(recommendations, indent = 4)
# # json_o3 = json.dumps(link, indent = 4)

# # Writing to sample.json
# with open("sample.txt", "w") as outfile:
#     outfile.write(json_object)
#     outfile.write(json_o2)
#     # outfile.write(json_o3)
# outfile.close()

In [ ]:
# # Write to file
# ofile = open("/content/gdrive/My Drive/Colab Notebooks/sample.json",mode='a')
# ofile.write(json_object)
# # outfile.a

391